In [6]:
import pandas as pd

df = pd.read_csv("./data/consumer_complaints.csv")  # update name if it's different
print(df.head())

  date_received           product               sub_product  \
0    08/30/2013          Mortgage            Other mortgage   
1    08/30/2013          Mortgage            Other mortgage   
2    08/30/2013  Credit reporting                       NaN   
3    08/30/2013      Student loan  Non-federal student loan   
4    08/30/2013   Debt collection               Credit card   

                                      issue  \
0  Loan modification,collection,foreclosure   
1  Loan servicing, payments, escrow account   
2    Incorrect information on credit report   
3                        Repaying your loan   
4        False statements or representation   

                           sub_issue consumer_complaint_narrative  \
0                                NaN                          NaN   
1                                NaN                          NaN   
2                     Account status                          NaN   
3                 Repaying your loan                          

/var/folders/sx/ffb2b5qj0psbzb1vdfjkxwd00000gn/T/ipykernel_3944/2839338843.py:3: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data/consumer_complaints.csv")  # update name if it's different


In [2]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# consumer_complaints.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('./data/consumer_complaints.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'consumer_complaints.csv'
nRow, nCol = df1.shape
print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 18 columns


In [4]:
from ollama import Client

client = Client(host="http://localhost:11434")  # default Ollama host

response = client.chat(
    model="deepseek-coder",
    messages=[{"role": "user", "content": "say hello"}]
)

print(response["message"]["content"])



Hello! How can I assist you with your programming or computer science needs today? Please feel free to ask any question related to these topics.



In [15]:
!pip install langchain langchain-community faiss-cpu sentence-transformers pandas
!pip install numpy==1.26.4 --force-reinstall


  Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-macosx_11_0_arm64.whl (13.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4


In [16]:
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document
import os


In [17]:
df = pd.read_csv("data/consumer_complaints.csv")

# Show a preview to verify it's loaded correctly
df.head()


/var/folders/sx/ffb2b5qj0psbzb1vdfjkxwd00000gn/T/ipykernel_3944/3779621301.py:1: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/consumer_complaints.csv")


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [18]:
# Make sure 'Consumer complaint narrative' exists
if "consumer_complaint_narrative" not in df.columns:
    raise ValueError("Expected 'Consumer complaint narrative' column not found!")

texts = df["consumer_complaint_narrative"].dropna().astype(str).tolist()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = splitter.create_documents(texts)

print(f"Number of documents: {len(documents)}")


Number of documents: 203643


In [23]:
import numpy
print(numpy.__version__)


2.2.5


In [24]:
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings

# Load model
llm = ChatOllama(model="deepseek-coder")

# Load vector store (assume you already created it)
vectorstore = FAISS.load_local("vectorstore_index", SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2"))

# Create RAG pipeline
qa = RetrievalQA.from_chain_type(llm=llm, retriever=vectorstore.as_retriever())

# Ask a question
response = qa.run("What are the top 3 complaint categories in the dataset?")
print(response)


/var/folders/sx/ffb2b5qj0psbzb1vdfjkxwd00000gn/T/ipykernel_3944/3234980318.py:7: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="deepseek-coder")


RuntimeError: Failed to import transformers.models.auto.modeling_auto because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
module 'numpy' has no attribute '_no_nep50_warning'